In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    AdaBoostRegressor
)
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,r2_score

from sklearn.decomposition import PCA

In [4]:
df = pd.read_csv('../../../data/processed/advance/gurgaon_properties_post_feature_selection_v2.csv')

In [5]:
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,0.82,3.0,2.0,2,New Property,850.0,0.0,0.0,0.0,Low,Low Floor
1,flat,sector 89,0.95,2.0,2.0,2,New Property,1226.0,1.0,0.0,0.0,Low,Mid Floor
2,flat,sohna road,0.32,2.0,2.0,1,New Property,1000.0,0.0,0.0,0.0,Low,High Floor
3,flat,sector 92,1.60,3.0,4.0,3+,Relatively New,1615.0,1.0,0.0,1.0,High,Mid Floor
4,flat,sector 102,0.48,2.0,2.0,1,Relatively New,582.0,0.0,1.0,0.0,High,Mid Floor


In [6]:
df['furnishing_type'].value_counts()

furnishing_type
0.0    2349
1.0    1018
2.0     187
Name: count, dtype: int64

In [7]:
# 0 -> unfurnished
# 1 -> semifurnished
# 2 -> furnished
df['furnishing_type'] = df['furnishing_type'].replace({0.0:'unfurnished',1.0:'semifurnished',2.0:'furnished'})

In [8]:
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,0.82,3.0,2.0,2,New Property,850.0,0.0,0.0,unfurnished,Low,Low Floor
1,flat,sector 89,0.95,2.0,2.0,2,New Property,1226.0,1.0,0.0,unfurnished,Low,Mid Floor
2,flat,sohna road,0.32,2.0,2.0,1,New Property,1000.0,0.0,0.0,unfurnished,Low,High Floor
3,flat,sector 92,1.60,3.0,4.0,3+,Relatively New,1615.0,1.0,0.0,semifurnished,High,Mid Floor
4,flat,sector 102,0.48,2.0,2.0,1,Relatively New,582.0,0.0,1.0,unfurnished,High,Mid Floor


In [9]:
X = df.drop(columns=['price'])
y = df['price']

In [10]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

### Ordinal Encoding

In [11]:
columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [14]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode)
    ], 
    remainder='passthrough'
)

In [15]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [16]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [17]:
scores.mean(),scores.std()

(np.float64(0.7363096633436828), np.float64(0.03238005754429938))

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [19]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'sector',
                                                   'balcony', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', LinearRegression())])

In [20]:
y_pred = pipeline.predict(X_test)

In [21]:
y_pred = np.expm1(y_pred)

In [22]:
mean_absolute_error(np.expm1(y_test),y_pred)

np.float64(0.9463822160089355)

In [23]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output
    

In [24]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [25]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [26]:
model_output

[['linear_reg',
  np.float64(0.7363096633436828),
  np.float64(0.9463822160089355)],
 ['svr', np.float64(0.7642012011196353), np.float64(0.8472636473483927)],
 ['ridge', np.float64(0.7363125343993554), np.float64(0.9463387741853388)],
 ['LASSO', np.float64(0.05943378064493572), np.float64(1.528905986892753)],
 ['decision tree',
  np.float64(0.7673085026151244),
  np.float64(0.7442615607081917)],
 ['random forest',
  np.float64(0.8821227799147175),
  np.float64(0.5328741406130693)],
 ['extra trees',
  np.float64(0.8669776195842113),
  np.float64(0.5484517439895076)],
 ['gradient boosting',
  np.float64(0.8726504776846961),
  np.float64(0.5759827420192102)],
 ['adaboost', np.float64(0.7538773267354032), np.float64(0.8319593806608996)],
 ['mlp', np.float64(0.8090576243465157), np.float64(0.7177366996634695)],
 ['xgboost', np.float64(0.8894876835260124), np.float64(0.5040475141482346)]]

In [27]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [28]:
model_df.sort_values(['mae'])

,name,r2,mae
10,xgboost,0.889488,0.504048
5,random forest,0.882123,0.532874
6,extra trees,0.866978,0.548452
7,gradient boosting,0.872650,0.575983
9,mlp,0.809058,0.717737
4,decision tree,0.767309,0.744262
8,adaboost,0.753877,0.831959
1,svr,0.764201,0.847264
2,ridge,0.736313,0.946339
0,linear_reg,0.736310,0.946382


### OneHotEncoding

In [29]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first'),['sector','agePossession','furnishing_type'])
    ], 
    remainder='passthrough'
)

In [30]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [31]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [32]:
scores.mean()

np.float64(0.8546054073648314)

In [33]:
scores.std()

np.float64(0.01599847663314007)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [35]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'sector',
                                                   'balcony', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category']),
                                                 ('cat1',
                                                  OneHotEncoder(drop='first'),
                                                  ['sector', 'agePossession',
                                                   'furnishing_type'])])),
                ('regressor', LinearRegression())])

In [36]:
y_pred = pipeline.predict(X_test)

In [37]:
y_pred = np.expm1(y_pred)

In [38]:
mean_absolute_error(np.expm1(y_test),y_pred)

np.float64(0.6497382874070646)

In [39]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output
    

In [40]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [41]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [42]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [43]:
model_df.sort_values(['mae'])

,name,r2,mae
6,extra trees,0.894422,0.466914
10,xgboost,0.895850,0.493456
5,random forest,0.890771,0.500038
9,mlp,0.874392,0.543562
7,gradient boosting,0.876294,0.570925
0,linear_reg,0.854605,0.649738
2,ridge,0.854678,0.652914
4,decision tree,0.807073,0.698884
1,svr,0.769741,0.834124
8,adaboost,0.751862,0.846813


### OneHotEncoding With PCA

In [47]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['sector','agePossession'])
    ], 
    remainder='passthrough'
)

In [48]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('pca', PCA(n_components=1)),
    ('regressor', LinearRegression())
])

In [49]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [19]:
scores.mean()

np.float64(0.06225201431451136)

In [20]:
scores.std()

np.float64(0.01986059407164015)

In [21]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('pca', PCA(n_components=0.95)),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output
    

In [22]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [23]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [24]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [25]:
model_df.sort_values(['mae'])

,name,r2,mae
5,random forest,0.762022,0.656978
6,extra trees,0.740420,0.695043
4,decision tree,0.696442,0.761509
10,xgboost,0.622205,0.967581
7,gradient boosting,0.610623,0.987906
1,svr,0.218073,1.361163
8,adaboost,0.305643,1.403254
9,mlp,0.208230,1.450977
2,ridge,0.062252,1.526707
0,linear_reg,0.062252,1.526707


In [66]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('pca', PCA()),
    ('regressor', LinearRegression())
])

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'sector',
                                                   'balcony', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category']),
                                                 ('cat1',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['sector',
                                                   'agePossession'])])),
                ('pca', PCA()), ('regressor', LinearRegression())])

In [68]:
y_pred = pipeline.predict(X_test)

In [69]:
y_pred = np.expm1(y_pred)

In [70]:
mean_absolute_error(np.expm1(y_test),y_pred)

np.float64(0.6491718597838264)

In [72]:
pipeline.named_steps['pca'].explained_variance_ratio_

array([9.89749744e-01, 2.90220572e-03, 1.68816536e-03, 9.31298980e-04,
       8.44026292e-04, 6.36740619e-04, 5.92022782e-04, 4.54962884e-04,
       4.17327781e-04, 2.82788570e-04, 1.87814840e-04, 1.64976967e-04,
       1.24921517e-04, 8.39921059e-05, 6.66087886e-05, 3.85820012e-05,
       2.76359720e-05, 2.58337502e-05, 2.43649092e-05, 2.42017556e-05,
       2.31068295e-05, 2.25533114e-05, 2.10023668e-05, 2.01997122e-05,
       1.87820810e-05, 1.74002386e-05, 1.69050844e-05, 1.63566237e-05,
       1.60336100e-05, 1.57538036e-05, 1.51270828e-05, 1.50071309e-05,
       1.49639547e-05, 1.42120409e-05, 1.41009031e-05, 1.38577795e-05,
       1.37957491e-05, 1.37128078e-05, 1.32341498e-05, 1.29527938e-05,
       1.28342844e-05, 1.22982822e-05, 1.21536402e-05, 1.16408767e-05,
       1.14468537e-05, 1.13839928e-05, 1.10374731e-05, 1.04931985e-05,
       1.04822944e-05, 1.01475504e-05, 9.93834155e-06, 9.76002650e-06,
       9.70271556e-06, 9.38192944e-06, 8.71443785e-06, 8.10680149e-06,
      

### Target Encoder

In [ ]:
!pip install category_encoders


[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import category_encoders as ce

In [55]:


columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ], 
    remainder='passthrough'
)

In [56]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [57]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [58]:
scores.mean(),scores.std()

(np.float64(0.829521918225536), np.float64(0.01838446337912282))

In [59]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output
    

In [60]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [61]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [62]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [63]:
model_df.sort_values(['mae'])

,name,r2,mae
10,xgboost,0.904798,0.447518
5,random forest,0.901154,0.452289
6,extra trees,0.901255,0.457758
7,gradient boosting,0.888952,0.507870
4,decision tree,0.827177,0.558826
9,mlp,0.849907,0.595438
8,adaboost,0.817599,0.707958
0,linear_reg,0.829522,0.713011
2,ridge,0.829536,0.713523
1,svr,0.782917,0.818851


### Hyperparameter Tuning

In [64]:
from sklearn.model_selection import GridSearchCV

In [65]:
param_grid = {
    'regressor__n_estimators': [50, 100, 200, 300],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__max_samples':[0.1, 0.25, 0.5, 1.0],
    'regressor__max_features': ['auto', 'sqrt']
}

In [66]:
columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ], 
    remainder='passthrough'
)

In [67]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

In [68]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

In [69]:
search = GridSearchCV(pipeline, param_grid, cv=kfold, scoring='r2', n_jobs=-1, verbose=4)

In [71]:
search.fit(X, y_transformed)

Fitting 10 folds for each of 128 candidates, totalling 1280 fits


c:\Users\vijit_singh\Desktop\Personal projects repo\Estate-Radar\env\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
640 fits failed out of a total of 1280.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
302 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\vijit_singh\Desktop\Personal projects repo\Estate-Radar\env\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\vijit_singh\Desktop\Personal projects repo\Estate-Radar\env\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^

GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['bedRoom',
                                                                          'bathroom',
                                                                          'built_up_area',
                                                                          'servant '
                                                                          'room',
                                                                          'store '
                                                                          'room']),
                                                                        ('cat',
                                                                         OrdinalEncoder(),
                                                                         ['property_type',
                                                                          'sector',
                                                                          'balcony',
                                                                          'agePossession',
                                                                          'furnishing_type',...
                                                                         OneHotEncoder(drop='first',
                                                                                       sparse_output=False),
                                                                         ['agePossession']),
                                                                        ('target_enc',
                                                                         TargetEncoder(),
                                                                         ['sector'])])),
                                       ('regressor', RandomForestRegressor())]),
             n_jobs=-1,
             param_grid={'regressor__max_depth': [None, 10, 20, 30],
                         'regressor__max_features': ['auto', 'sqrt'],
                         'regressor__max_samples': [0.1, 0.25, 0.5, 1.0],
                         'regressor__n_estimators': [50, 100, 200, 300]},
             scoring='r2', verbose=4)

In [72]:
final_pipe = search.best_estimator_

In [73]:
search.best_params_

{'regressor__max_depth': 30,
 'regressor__max_features': 'sqrt',
 'regressor__max_samples': 1.0,
 'regressor__n_estimators': 300}

In [74]:
search.best_score_

np.float64(0.9026121231171291)

In [75]:
final_pipe.fit(X,y_transformed)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'sector',
                                                   'balcony', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category']),
                                                 ('cat1',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['agePossession']),
                                                 ('target_enc', TargetEncoder(),
                                                  ['sector'])])),
                ('regressor',
                 RandomForestRegressor(max_depth=30, max_features='sqrt',
                                       max_samples=1.0, n_estimators=300))])

### Exporting the model

In [76]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['sector','agePossession'])
    ], 
    remainder='passthrough'
)

In [77]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=500))
])

In [78]:
pipeline.fit(X,y_transformed)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'sector',
                                                   'balcony', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category']),
                                                 ('cat1',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['sector',
                                                   'agePossession'])])),
                ('regressor', RandomForestRegressor(n_estimators=500))])

In [80]:
import pickle

with open('../../../models/pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [89]:
with open('../../../models/df.pkl', 'wb') as file:
    pickle.dump(X, file)

In [84]:
X

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,3.0,2.0,2,New Property,850.0,0.0,0.0,unfurnished,Low,Low Floor
1,flat,sector 89,2.0,2.0,2,New Property,1226.0,1.0,0.0,unfurnished,Low,Mid Floor
2,flat,sohna road,2.0,2.0,1,New Property,1000.0,0.0,0.0,unfurnished,Low,High Floor
3,flat,sector 92,3.0,4.0,3+,Relatively New,1615.0,1.0,0.0,semifurnished,High,Mid Floor
4,flat,sector 102,2.0,2.0,1,Relatively New,582.0,0.0,1.0,unfurnished,High,Mid Floor
...,...,...,...,...,...,...,...,...,...,...,...,...
3549,flat,sector 84,2.0,2.0,1,Relatively New,532.0,0.0,0.0,unfurnished,Medium,Mid Floor
3550,house,sector 109,5.0,5.0,3+,Relatively New,6228.0,1.0,1.0,unfurnished,High,Low Floor
3551,flat,sector 2,1.0,1.0,1,Moderately Old,665.0,0.0,0.0,semifurnished,Medium,Mid Floor
3552,house,sector 43,5.0,6.0,3,Moderately Old,5490.0,1.0,1.0,unfurnished,Medium,Mid Floor


### Trying out the predictions

In [85]:
X.columns

Index(['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category'],
      dtype='object')

In [86]:
X.iloc[0].values

array(['flat', 'sector 36', np.float64(3.0), np.float64(2.0), '2',
       'New Property', np.float64(850.0), np.float64(0.0),
       np.float64(0.0), 'unfurnished', 'Low', 'Low Floor'], dtype=object)

In [87]:
data = [['house', 'sector 102', 4, 3, '3+', 'New Property', 2750, 0, 0, 'unfurnished', 'Low', 'Low Floor']]
columns = ['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category']

# Convert to DataFrame
one_df = pd.DataFrame(data, columns=columns)

one_df


,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,house,sector 102,4,3,3+,New Property,2750,0,0,unfurnished,Low,Low Floor


In [88]:
np.expm1(pipeline.predict(one_df))

array([3.19052593])

In [307]:
X.dtypes

property_type       object
sector              object
bedRoom            float64
bathroom           float64
balcony             object
agePossession       object
built_up_area      float64
servant room       float64
store room         float64
furnishing_type     object
luxury_category     object
floor_category      object
dtype: object

In [306]:
sorted(X['sector'].unique().tolist())

['dwarka expressway',
 'gwal pahari',
 'manesar',
 'sector 1',
 'sector 10',
 'sector 102',
 'sector 103',
 'sector 104',
 'sector 105',
 'sector 106',
 'sector 107',
 'sector 108',
 'sector 109',
 'sector 11',
 'sector 110',
 'sector 111',
 'sector 112',
 'sector 113',
 'sector 12',
 'sector 13',
 'sector 14',
 'sector 15',
 'sector 17',
 'sector 2',
 'sector 21',
 'sector 22',
 'sector 23',
 'sector 24',
 'sector 25',
 'sector 26',
 'sector 27',
 'sector 28',
 'sector 3',
 'sector 30',
 'sector 31',
 'sector 33',
 'sector 36',
 'sector 37',
 'sector 37d',
 'sector 38',
 'sector 39',
 'sector 4',
 'sector 40',
 'sector 41',
 'sector 43',
 'sector 45',
 'sector 46',
 'sector 47',
 'sector 48',
 'sector 49',
 'sector 5',
 'sector 50',
 'sector 51',
 'sector 52',
 'sector 53',
 'sector 54',
 'sector 55',
 'sector 56',
 'sector 57',
 'sector 58',
 'sector 59',
 'sector 6',
 'sector 60',
 'sector 61',
 'sector 62',
 'sector 63',
 'sector 63a',
 'sector 65',
 'sector 66',
 'sector 67',
 'se

### Optuna

In [11]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

c:\Users\vijit_singh\Desktop\Personal projects repo\Estate-Radar\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
! pip install optuna-integration[xgboost]

In [ ]:
def objective(trial):
    # Define the preprocessor
    columns_to_encode = ['property_type', 'sector', 'balcony', 'agePossession', 
                        'furnishing_type', 'luxury_category', 'floor_category']
    
    preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ], 
    remainder='passthrough'
    )

    # Choose the algorithm to tune
    regressor_name = trial.suggest_categorical('regressor', ['RandomForest', 'GradientBoosting', 'XGBoost'])

    if regressor_name == 'RandomForest':
        model = RandomForestRegressor(
            n_estimators=trial.suggest_int('n_estimators', 100, 500),
            max_depth=trial.suggest_int('max_depth', 5, 30),
            min_samples_split=trial.suggest_int('min_samples_split', 2, 10),
            min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 10),
            random_state=42
        )

    elif regressor_name == 'GradientBoosting':
        model = GradientBoostingRegressor(
            n_estimators=trial.suggest_int('n_estimators', 100, 500),
            learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            max_depth=trial.suggest_int('max_depth', 3, 20),
            min_samples_split=trial.suggest_int('min_samples_split', 2, 10),
            min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 10),
            random_state=42
        )

    else:  # XGBoost
        model = XGBRegressor(
            n_estimators=trial.suggest_int('n_estimators', 100, 500),
            max_depth=trial.suggest_int('max_depth', 3, 20),
            learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            min_child_weight=trial.suggest_int('min_child_weight', 1, 7),
            subsample=trial.suggest_float('subsample', 0.6, 1.0),
            colsample_bytree=trial.suggest_float('colsample_bytree', 0.6, 1.0),
            random_state=42
        )

    # Create pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    try:
        # Custom scorer for MAE on original scale
        def mae_scorer(estimator, X, y):
            y_pred = estimator.predict(X)
            return -mean_absolute_error(np.expm1(y), np.expm1(y_pred))  # Negative because we want to maximize
        
        # Perform cross-validation
        kfold = KFold(n_splits=5, shuffle=True, random_state=42)
        
        # Get both R2 and MAE scores
        r2_scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
        mae_scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring=mae_scorer)
        
        mean_r2 = r2_scores.mean()
        mean_mae = -mae_scores.mean()  # Convert back to positive
        
        print(f"\nTrial Results:")
        print(f"Mean R2 Score: {mean_r2:.4f} (+/- {r2_scores.std() * 2:.4f})")
        print(f"Mean MAE: {mean_mae:.4f} (+/- {mae_scores.std() * 2:.4f})")
        
        return mean_mae
        
    except Exception as e:
        print(f"Error during evaluation: {str(e)}")
        return float('inf')

# Create and run the study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50, n_jobs=-1)

# Print the best results
print("\nBest trial:")
print("  MAE: ", study.best_trial.value)
print("  Params: ")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

In [19]:
# First, do the train-test split outside the objective function
X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.2, random_state=42)

def objective(trial):
    # Define the preprocessor
    columns_to_encode = ['property_type', 'sector', 'balcony', 'agePossession', 
                        'furnishing_type', 'luxury_category', 'floor_category']
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
            ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), columns_to_encode),
            ('cat1', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), ['agePossession']),
            ('target_enc', ce.TargetEncoder(), ['sector'])
    ], 
    remainder='passthrough'
    )

    # XGBoost hyperparameters - expanded search space
    model = XGBRegressor(
        # Tree-specific parameters
        n_estimators=trial.suggest_int('n_estimators', 100, 1000),
        max_depth=trial.suggest_int('max_depth', 3, 25),
        min_child_weight=trial.suggest_int('min_child_weight', 1, 10),
        
        # Learning rate and regularization
        learning_rate=trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
        gamma=trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        reg_alpha=trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        reg_lambda=trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
        
        # Sampling parameters
        subsample=trial.suggest_float('subsample', 0.5, 1.0),
        colsample_bytree=trial.suggest_float('colsample_bytree', 0.5, 1.0),
        colsample_bylevel=trial.suggest_float('colsample_bylevel', 0.5, 1.0),
        colsample_bynode=trial.suggest_float('colsample_bynode', 0.5, 1.0),
        
        # Other parameters
        max_delta_step=trial.suggest_int('max_delta_step', 0, 10),
        scale_pos_weight=trial.suggest_float('scale_pos_weight', 0.8, 1.2),
        
        # Fixed parameters
        random_state=42,
        n_jobs=-1,
        tree_method='hist'
    )

    # Create pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    try:
        # Use K-fold CV on training data only
        kfold = KFold(n_splits=5, shuffle=True, random_state=42)
        
        # Get cross-validation scores on training data
        cv_scores_r2 = []
        cv_scores_mae = []
        
        for train_idx, val_idx in kfold.split(X_train):
            # Split the training data into train and validation
            X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
            
            # Fit the pipeline on training fold
            pipeline.fit(X_train_fold, y_train_fold)
            
            # Make predictions on validation fold
            y_pred_fold = pipeline.predict(X_val_fold)
            
            # Calculate metrics
            r2 = r2_score(y_val_fold, y_pred_fold)
            mae = mean_absolute_error(np.expm1(y_val_fold), np.expm1(y_pred_fold))
            
            cv_scores_r2.append(r2)
            cv_scores_mae.append(mae)
        
        mean_r2 = np.mean(cv_scores_r2)
        mean_mae = np.mean(cv_scores_mae)
        
        print(f"\nTrial Results:")
        print(f"Mean R2 Score: {mean_r2:.4f} (+/- {np.std(cv_scores_r2) * 2:.4f})")
        print(f"Mean MAE: {mean_mae:.4f} (+/- {np.std(cv_scores_mae) * 2:.4f})")
        
        return mean_mae
        
    except Exception as e:
        print(f"Error during evaluation: {str(e)}")
        return float('inf')

# Create and run the study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, n_jobs=-1)

# Get the best pipeline
best_params = study.best_trial.params
best_pipeline = Pipeline([
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
            ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), columns_to_encode),
            ('cat1', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), ['agePossession'])
        ], 
        remainder='passthrough'
    )),
    ('regressor', XGBRegressor(**best_params, random_state=42, n_jobs=-1, tree_method='hist'))
])

# Fit the best pipeline on the full training data
best_pipeline.fit(X_train, y_train)

# Evaluate on test set
y_pred_test = best_pipeline.predict(X_test)
final_r2 = r2_score(y_test, y_pred_test)
final_mae = mean_absolute_error(np.expm1(y_test), np.expm1(y_pred_test))

print("\nBest trial:")
print("  MAE: ", study.best_trial.value)
print("\nBest hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

print("\nFinal Test Set Performance:")
print(f"R2 Score: {final_r2:.4f}")
print(f"MAE: {final_mae:.4f}")

# Plot optimization history
try:
    optuna.visualization.plot_optimization_history(study)
    optuna.visualization.plot_param_importances(study)
except:
    print("Visualization requires plotly to be installed")

[I 2024-12-19 17:53:28,280] A new study created in memory with name: no-name-af39cc82-28a3-4692-be75-c57c3626b2ed
[I 2024-12-19 17:53:38,571] Trial 2 finished with value: 0.5210861573772798 and parameters: {'n_estimators': 668, 'max_depth': 13, 'min_child_weight': 9, 'learning_rate': 0.11066622389083881, 'gamma': 0.03715246886984261, 'reg_alpha': 0.02791000086457536, 'reg_lambda': 0.0006982515207901851, 'subsample': 0.7505168621972407, 'colsample_bytree': 0.575841481611983, 'colsample_bylevel': 0.5366001890133405, 'colsample_bynode': 0.9470682401238679, 'max_delta_step': 1, 'scale_pos_weight': 1.0794545856347113}. Best is trial 2 with value: 0.5210861573772798.



Trial Results:
Mean R2 Score: 0.8934 (+/- 0.0145)
Mean MAE: 0.5211 (+/- 0.0779)


[I 2024-12-19 17:53:40,622] Trial 6 finished with value: 0.5393645798834242 and parameters: {'n_estimators': 632, 'max_depth': 3, 'min_child_weight': 3, 'learning_rate': 0.02023902150981271, 'gamma': 0.000539739680341382, 'reg_alpha': 1.3232361013323386e-07, 'reg_lambda': 0.19423287299196917, 'subsample': 0.5615983104864994, 'colsample_bytree': 0.6062295717737727, 'colsample_bylevel': 0.8454719962796674, 'colsample_bynode': 0.799011081354126, 'max_delta_step': 6, 'scale_pos_weight': 1.1112846087871042}. Best is trial 2 with value: 0.5210861573772798.



Trial Results:
Mean R2 Score: 0.8884 (+/- 0.0216)
Mean MAE: 0.5394 (+/- 0.0876)


[I 2024-12-19 17:53:51,478] Trial 5 finished with value: 0.5186115845229715 and parameters: {'n_estimators': 481, 'max_depth': 6, 'min_child_weight': 8, 'learning_rate': 0.15481798102127978, 'gamma': 0.00011036137695867412, 'reg_alpha': 0.4361073118688641, 'reg_lambda': 4.15083856753968e-06, 'subsample': 0.580846146488123, 'colsample_bytree': 0.572302904536752, 'colsample_bylevel': 0.5646934493267055, 'colsample_bynode': 0.6846302772120175, 'max_delta_step': 4, 'scale_pos_weight': 1.1363892744312902}. Best is trial 5 with value: 0.5186115845229715.


...


Trial Results:
Mean R2 Score: 0.9008 (+/- 0.0177)
Mean MAE: 0.4933 (+/- 0.0821)


[I 2024-12-19 18:01:58,329] Trial 97 finished with value: 0.48876666795864765 and parameters: {'n_estimators': 738, 'max_depth': 25, 'min_child_weight': 7, 'learning_rate': 0.011308454519366918, 'gamma': 1.236796847760128e-07, 'reg_alpha': 3.630404592799856e-05, 'reg_lambda': 1.2671882776238324e-06, 'subsample': 0.9062884841198149, 'colsample_bytree': 0.7535870016840406, 'colsample_bylevel': 0.936582508813245, 'colsample_bynode': 0.6286603358601679, 'max_delta_step': 0, 'scale_pos_weight': 0.9227318480515998}. Best is trial 92 with value: 0.48599784767341714.



Trial Results:
Mean R2 Score: 0.9032 (+/- 0.0173)
Mean MAE: 0.4888 (+/- 0.0804)


[I 2024-12-19 18:02:00,312] Trial 99 finished with value: 0.4890745436722409 and parameters: {'n_estimators': 743, 'max_depth': 25, 'min_child_weight': 7, 'learning_rate': 0.011459734748772531, 'gamma': 1.439419158513737e-07, 'reg_alpha': 7.71174751101627e-05, 'reg_lambda': 1.1545324517572843e-06, 'subsample': 0.8212224370969629, 'colsample_bytree': 0.8181248457403625, 'colsample_bylevel': 0.9814972977749811, 'colsample_bynode': 0.6214880415773358, 'max_delta_step': 1, 'scale_pos_weight': 0.854235669584738}. Best is trial 92 with value: 0.48599784767341714.



Trial Results:
Mean R2 Score: 0.9026 (+/- 0.0190)
Mean MAE: 0.4891 (+/- 0.0803)

Best trial:
  MAE:  0.48599784767341714

Best hyperparameters:
    n_estimators: 890
    max_depth: 22
    min_child_weight: 6
    learning_rate: 0.013894939430122396
    gamma: 8.850284671073111e-07
    reg_alpha: 4.702724569459745e-06
    reg_lambda: 0.06820048029803732
    subsample: 0.8940609379239972
    colsample_bytree: 0.8102952066278133
    colsample_bylevel: 0.6912472726243479
    colsample_bynode: 0.627282584000448
    max_delta_step: 0
    scale_pos_weight: 0.8874979346935884

Final Test Set Performance:
R2 Score: 0.9111
MAE: 0.4934


In [21]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.2, random_state=42)

# Define preprocessing
numeric_features = ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']
categorical_features = ['property_type', 'sector', 'balcony', 'agePossession', 
                       'furnishing_type', 'luxury_category', 'floor_category']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), 
         categorical_features)
    ]
)

def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'eval_metric': 'mae',
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 25),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.4, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.4, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000)
    }

    # Preprocess the data
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)
    
    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train_processed, label=y_train)
    dtest = xgb.DMatrix(X_test_processed, label=y_test)

    # Define pruning callback
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mae")

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=param['n_estimators'],
        evals=[(dtrain, "train"), (dtest, "eval")],
        early_stopping_rounds=50,
        callbacks=[pruning_callback]
    )

    # Predict on test set
    preds = bst.predict(dtest)
    
    # Calculate metrics
    mae = mean_absolute_error(np.expm1(y_test), np.expm1(preds))
    r2 = r2_score(y_test, preds)
    
    print(f"\nTrial Results:")
    print(f"MAE: {mae:.4f}")
    print(f"R2 Score: {r2:.4f}")
    
    return mae

# Create and run study
study = optuna.create_study(
    direction='minimize',
    pruner=optuna.pruners.SuccessiveHalvingPruner()
)
study.optimize(objective, n_trials=100)

# Train final model with best parameters
best_params = study.best_trial.params
best_params['objective'] = 'reg:squarederror'
best_params['eval_metric'] = 'mae'

# Preprocess final data
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Create and train final model
dtrain = xgb.DMatrix(X_train_processed, label=y_train)
dtest = xgb.DMatrix(X_test_processed, label=y_test)
final_model = xgb.train(
    best_params,
    dtrain,
    num_boost_round=best_params['n_estimators'],
    early_stopping_rounds=50,
    evals=[(dtrain, "train"), (dtest, "eval")]
)

# Final evaluation
final_preds = final_model.predict(dtest)
final_mae = mean_absolute_error(np.expm1(y_test), np.expm1(final_preds))
final_r2 = r2_score(y_test, final_preds)

print("\nBest trial:")
print("  MAE: ", study.best_trial.value)
print("\nBest hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

print("\nFinal Model Performance:")
print(f"MAE: {final_mae:.4f}")
print(f"R2 Score: {final_r2:.4f}")

# Visualization
try:
    import plotly
    fig1 = optuna.visualization.plot_optimization_history(study)
    fig2 = optuna.visualization.plot_param_importances(study)
    fig1.show()
    fig2.show()
except:
    print("Visualization requires plotly to be installed")

[I 2024-12-19 20:29:38,360] A new study created in memory with name: no-name-00228131-4dd5-4b35-a268-64a7a8ab193e


[0]	train-mae:0.38079	eval-mae:0.38673
[1]	train-mae:0.35410	eval-mae:0.36117
[2]	train-mae:0.31490	eval-mae:0.32599
[3]	train-mae:0.28561	eval-mae:0.29834
[4]	train-mae:0.25807	eval-mae:0.27260
[5]	train-mae:0.23939	eval-mae:0.25437
[6]	train-mae:0.21840	eval-mae:0.23728
[7]	train-mae:0.20000	eval-mae:0.22006
[8]	train-mae:0.18384	eval-mae:0.20548
[9]	train-mae:0.16832	eval-mae:0.19196
[10]	train-mae:0.15617	eval-mae:0.18319
[11]	train-mae:0.14424	eval-mae:0.17375
[12]	train-mae:0.13371	eval-mae:0.16617
[13]	train-mae:0.12406	eval-mae:0.16014
[14]	train-mae:0.11693	eval-mae:0.15603
[15]	train-mae:0.11327	eval-mae:0.15435
[16]	train-mae:0.10741	eval-mae:0.15162
[17]	train-mae:0.10147	eval-mae:0.14850
[18]	train-mae:0.09530	eval-mae:0.14562
[19]	train-mae:0.08933	eval-mae:0.14186
[20]	train-mae:0.08634	eval-mae:0.14015
[21]	train-mae:0.08302	eval-mae:0.13811
[22]	train-mae:0.07823	eval-mae:0.13544
[23]	train-mae:0.07610	eval-mae:0.13487
[24]	train-mae:0.07274	eval-mae:0.13379
[25]	train

[I 2024-12-19 20:29:39,715] Trial 0 finished with value: 0.5365749346991966 and parameters: {'lambda': 3.208553013867788e-05, 'alpha': 2.954581248925638e-06, 'eta': 0.12868103454866872, 'gamma': 1.72299590537457e-05, 'max_depth': 25, 'min_child_weight': 3, 'subsample': 0.9329615962263436, 'colsample_bytree': 0.697687903627443, 'colsample_bylevel': 0.440669602906547, 'colsample_bynode': 0.8397072686133679, 'n_estimators': 669}. Best is trial 0 with value: 0.5365749346991966.



Trial Results:
MAE: 0.5366
R2 Score: 0.8976
[0]	train-mae:0.33646	eval-mae:0.34922
[1]	train-mae:0.26862	eval-mae:0.28252
[2]	train-mae:0.22047	eval-mae:0.24125
[3]	train-mae:0.18068	eval-mae:0.20609
[4]	train-mae:0.15029	eval-mae:0.18008
[5]	train-mae:0.12739	eval-mae:0.16341
[6]	train-mae:0.11117	eval-mae:0.15213
[7]	train-mae:0.09788	eval-mae:0.14324
[8]	train-mae:0.08931	eval-mae:0.13937
[9]	train-mae:0.08164	eval-mae:0.13586
[10]	train-mae:0.07587	eval-mae:0.13377
[11]	train-mae:0.07035	eval-mae:0.13083
[12]	train-mae:0.06511	eval-mae:0.12855
[13]	train-mae:0.06154	eval-mae:0.12675
[14]	train-mae:0.05863	eval-mae:0.12532
[15]	train-mae:0.05590	eval-mae:0.12469
[16]	train-mae:0.05366	eval-mae:0.12471
[17]	train-mae:0.05154	eval-mae:0.12452
[18]	train-mae:0.04976	eval-mae:0.12427
[19]	train-mae:0.04796	eval-mae:0.12437
[20]	train-mae:0.04678	eval-mae:0.12445
[21]	train-mae:0.04493	eval-mae:0.12425
[22]	train-mae:0.04374	eval-mae:0.12440
[23]	train-mae:0.04243	eval-mae:0.12434
[24]	

[I 2024-12-19 20:29:40,563] Trial 1 finished with value: 0.5278477369280304 and parameters: {'lambda': 2.315389887198197e-08, 'alpha': 1.2486122404739032e-07, 'eta': 0.24328128641642952, 'gamma': 0.00013553911078582632, 'max_depth': 15, 'min_child_weight': 4, 'subsample': 0.7861633757353756, 'colsample_bytree': 0.9480942783433611, 'colsample_bylevel': 0.5429772823826569, 'colsample_bynode': 0.8584820253634963, 'n_estimators': 625}. Best is trial 1 with value: 0.5278477369280304.


...

[0]	train-mae:0.32141	eval-mae:0.33347
[1]	train-mae:0.25029	eval-mae:0.26584
[2]	train-mae:0.19412	eval-mae:0.21397
[3]	train-mae:0.15410	eval-mae:0.18127
[4]	train-mae:0.12571	eval-mae:0.15701
[5]	train-mae:0.10538	eval-mae:0.14299
[6]	train-mae:0.09114	eval-mae:0.13465
[7]	train-mae:0.08068	eval-mae:0.13019
[8]	train-mae:0.07254	eval-mae:0.12719
[9]	train-mae:0.06769	eval-mae:0.12596
[10]	train-mae:0.06327	eval-mae:0.12491
[11]	train-mae:0.05991	eval-mae:0.12386
[12]	train-mae:0.05758	eval-mae:0.12373
[13]	train-mae:0.05569	eval-mae:0.12332
[14]	train-mae:0.05422	eval-mae:0.12329
[15]	train-mae:0.05208	eval-mae:0.12328


[I 2024-12-19 20:30:01,266] Trial 98 pruned. Trial was pruned at iteration 16.


[0]	train-mae:0.32254	eval-mae:0.32692
[1]	train-mae:0.25683	eval-mae:0.26469
[2]	train-mae:0.21088	eval-mae:0.22293
[3]	train-mae:0.17460	eval-mae:0.19062


[I 2024-12-19 20:30:01,451] Trial 99 pruned. Trial was pruned at iteration 4.


[0]	train-mae:0.31759	eval-mae:0.33023
[1]	train-mae:0.24130	eval-mae:0.26364
[2]	train-mae:0.18157	eval-mae:0.21452
[3]	train-mae:0.13767	eval-mae:0.17880
[4]	train-mae:0.10751	eval-mae:0.15901
[5]	train-mae:0.08356	eval-mae:0.14322
[6]	train-mae:0.06561	eval-mae:0.13361
[7]	train-mae:0.05260	eval-mae:0.12784
[8]	train-mae:0.04297	eval-mae:0.12495
[9]	train-mae:0.03552	eval-mae:0.12250
[10]	train-mae:0.03039	eval-mae:0.12087
[11]	train-mae:0.02648	eval-mae:0.11968
[12]	train-mae:0.02267	eval-mae:0.11870
[13]	train-mae:0.02070	eval-mae:0.11859
[14]	train-mae:0.01890	eval-mae:0.11831
[15]	train-mae:0.01709	eval-mae:0.11799
[16]	train-mae:0.01585	eval-mae:0.11768
[17]	train-mae:0.01504	eval-mae:0.11749
[18]	train-mae:0.01470	eval-mae:0.11744
[19]	train-mae:0.01402	eval-mae:0.11733
[20]	train-mae:0.01380	eval-mae:0.11726
[21]	train-mae:0.01350	eval-mae:0.11718
[22]	train-mae:0.01318	eval-mae:0.11727
[23]	train-mae:0.01305	eval-mae:0.11729
[24]	train-mae:0.01305	eval-mae:0.11729
[25]	train

In [23]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
import category_encoders as ce

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.2, random_state=42)

def create_features(X):
    """Create additional features"""
    X = X.copy()
    
    # Area per bedroom
    X['area_per_bedroom'] = X['built_up_area'] / (X['bedRoom'] + 1)
    
    # Bathroom to bedroom ratio
    X['bath_bed_ratio'] = X['bathroom'] / (X['bedRoom'] + 1)
    
    # Total rooms
    X['total_rooms'] = X['bedRoom'] + X['bathroom'] + X['servant room'] + X['store room']
    
    return X

def preprocess_data(X_train, X_val, y_train):
    """Preprocess data with proper target encoding"""
    # Define features by type
    numeric_features = ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room', 
                       'area_per_bedroom', 'bath_bed_ratio', 'total_rooms']
    categorical_features = ['property_type', 'balcony', 'luxury_category', 'floor_category']
    target_encode_features = ['sector']
    onehot_features = ['agePossession', 'furnishing_type']
    
    # Scale numeric features
    scaler = StandardScaler()
    X_train_num = scaler.fit_transform(X_train[numeric_features])
    X_val_num = scaler.transform(X_val[numeric_features])
    
    # Ordinal encode basic categorical features
    ord_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    X_train_cat = ord_enc.fit_transform(X_train[categorical_features])
    X_val_cat = ord_enc.transform(X_val[categorical_features])
    
    # Target encode high cardinality features
    target_enc = ce.TargetEncoder(smoothing=0.5)
    X_train_target = target_enc.fit_transform(X_train[target_encode_features], y_train)
    X_val_target = target_enc.transform(X_val[target_encode_features])
    
    # One-hot encode appropriate features
    onehot_enc = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
    X_train_onehot = onehot_enc.fit_transform(X_train[onehot_features])
    X_val_onehot = onehot_enc.transform(X_val[onehot_features])
    
    # Combine all features
    X_train_processed = np.hstack([X_train_num, X_train_cat, X_train_target, X_train_onehot])
    X_val_processed = np.hstack([X_val_num, X_val_cat, X_val_target, X_val_onehot])
    
    return X_train_processed, X_val_processed

def objective(trial):
    # Create features
    X_train_feat = create_features(X_train)
    X_test_feat = create_features(X_test)
    
    # Enhanced hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'eval_metric': 'mae',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        'lambda': trial.suggest_float('lambda', 1e-8, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 10.0, log=True),
        'eta': trial.suggest_float('eta', 0.001, 0.5, log=True),
        'gamma': trial.suggest_float('gamma', 1e-8, 10.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 30),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 15),
        'subsample': trial.suggest_float('subsample', 0.3, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.3, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.3, 1.0),
        'max_delta_step': trial.suggest_int('max_delta_step', 0, 10),
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
    }
    
    if param['booster'] == 'dart':
        param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
        param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
        param['rate_drop'] = trial.suggest_float('rate_drop', 0.0, 0.5)
        param['skip_drop'] = trial.suggest_float('skip_drop', 0.0, 0.5)
    
    # Implement k-fold cross validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mae_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_feat)):
        X_fold_train = X_train_feat.iloc[train_idx]
        y_fold_train = y_train.iloc[train_idx]
        X_fold_val = X_train_feat.iloc[val_idx]
        y_fold_val = y_train.iloc[val_idx]
        
        # Preprocess the data
        X_fold_train_processed, X_fold_val_processed = preprocess_data(
            X_fold_train, X_fold_val, y_fold_train
        )
        
        # Create DMatrix
        dtrain = xgb.DMatrix(X_fold_train_processed, label=y_fold_train)
        dval = xgb.DMatrix(X_fold_val_processed, label=y_fold_val)
        
        # Train model
        model = xgb.train(
            param,
            dtrain,
            num_boost_round=param['n_estimators'],
            evals=[(dval, 'val')],
            early_stopping_rounds=50,
            verbose_eval=False
        )
        
        # Predict and calculate MAE
        preds = model.predict(dval)
        mae = mean_absolute_error(np.expm1(y_fold_val), np.expm1(preds))
        mae_scores.append(mae)
    
    mean_mae = np.mean(mae_scores)
    mae_std = np.std(mae_scores)
    final_score = mean_mae * (1 + mae_std)
    
    trial.set_user_attr('mae_std', mae_std)
    trial.set_user_attr('raw_mae', mean_mae)
    
    return final_score

# Create and run study
study = optuna.create_study(
    direction='minimize',
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=5,
        n_warmup_steps=5,
        interval_steps=3
    )
)
study.optimize(objective, n_trials=150, n_jobs=-1)

# Train final model with best parameters
best_params = study.best_trial.params
best_params['objective'] = 'reg:squarederror'
best_params['eval_metric'] = 'mae'

# Create final features and preprocess
X_train_feat = create_features(X_train)
X_test_feat = create_features(X_test)
X_train_processed, X_test_processed = preprocess_data(X_train_feat, X_test_feat, y_train)

# Train final model
dtrain = xgb.DMatrix(X_train_processed, label=y_train)
dtest = xgb.DMatrix(X_test_processed, label=y_test)
final_model = xgb.train(
    best_params,
    dtrain,
    num_boost_round=best_params['n_estimators'],
    early_stopping_rounds=50,
    evals=[(dtrain, "train"), (dtest, "eval")]
)

# Final evaluation
final_preds = final_model.predict(dtest)
final_mae = mean_absolute_error(np.expm1(y_test), np.expm1(final_preds))
final_r2 = r2_score(y_test, final_preds)

print("\nBest trial:")
print("  MAE: ", study.best_trial.value)
print("  Raw MAE: ", study.best_trial.user_attrs['raw_mae'])
print("  MAE Std: ", study.best_trial.user_attrs['mae_std'])
print("\nBest hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

print("\nFinal Model Performance:")
print(f"MAE: {final_mae:.4f}")
print(f"R2 Score: {final_r2:.4f}")

# Feature importance analysis
feature_importance = final_model.get_score(importance_type='gain')
sorted_importance = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
print("\nTop 10 Most Important Features:")
for feat, importance in sorted_importance[:10]:
    print(f"{feat}: {importance:.4f}")

# Visualization
try:
    import plotly
    fig1 = optuna.visualization.plot_optimization_history(study)
    fig2 = optuna.visualization.plot_param_importances(study)
    fig1.show()
    fig2.show()
except:
    print("Visualization requires plotly to be installed")

[I 2024-12-19 20:34:27,114] A new study created in memory with name: no-name-c064aa33-7e5a-451a-aba4-bc9bb3c180d2


[I 2024-12-19 20:34:32,351] Trial 0 finished with value: 0.8326912323154253 and parameters: {'booster': 'gbtree', 'lambda': 1.109207787001655e-06, 'alpha': 0.007464261249737968, 'eta': 0.014732531528239067, 'gamma': 7.4397005941871726, 'max_depth': 25, 'min_child_weight': 14, 'subsample': 0.5611315552240134, 'colsample_bytree': 0.6331288833127087, 'colsample_bylevel': 0.641447113322807, 'colsample_bynode': 0.7920628117065205, 'max_delta_step': 9, 'n_estimators': 232, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.8326912323154253.
[I 2024-12-19 20:34:40,354] Trial 1 finished with value: 0.5550641196726105 and parameters: {'booster': 'gbtree', 'lambda': 8.623053810891996e-05, 'alpha': 0.04093499007577015, 'eta': 0.049886074707448075, 'gamma': 0.008764101637395664, 'max_depth': 12, 'min_child_weight': 9, 'subsample': 0.9188721168944667, 'colsample_bytree': 0.9254590100510736, 'colsample_bylevel': 0.7153834068139265, 'colsample_bynode': 0.7923041215316966, 'max_delta_step': 9,

[0]	train-mae:0.42304	eval-mae:0.42809
[1]	train-mae:0.42015	eval-mae:0.42520
[2]	train-mae:0.41686	eval-mae:0.42199
[3]	train-mae:0.41352	eval-mae:0.41862
[4]	train-mae:0.40988	eval-mae:0.41508
[5]	train-mae:0.40635	eval-mae:0.41160
[6]	train-mae:0.40307	eval-mae:0.40839
[7]	train-mae:0.39959	eval-mae:0.40498
[8]	train-mae:0.39619	eval-mae:0.40159
[9]	train-mae:0.39305	eval-mae:0.39847
[10]	train-mae:0.39000	eval-mae:0.39553
[11]	train-mae:0.38674	eval-mae:0.39222
[12]	train-mae:0.38347	eval-mae:0.38893
[13]	train-mae:0.38014	eval-mae:0.38572
[14]	train-mae:0.37724	eval-mae:0.38289
[15]	train-mae:0.37444	eval-mae:0.38012
[16]	train-mae:0.37195	eval-mae:0.37768
[17]	train-mae:0.36918	eval-mae:0.37505
[18]	train-mae:0.36605	eval-mae:0.37187
[19]	train-mae:0.36305	eval-mae:0.36894
[20]	train-mae:0.35996	eval-mae:0.36585
[21]	train-mae:0.35692	eval-mae:0.36291
[22]	train-mae:0.35412	eval-mae:0.36019
[23]	train-mae:0.35138	eval-mae:0.35753
[24]	train-mae:0.34916	eval-mae:0.35532
[25]	train

In [24]:
# Best MAE (includes variance penalty)
print("Best MAE (with variance penalty):", study.best_trial.value)

# Raw MAE (without variance penalty)
print("Raw MAE:", study.best_trial.user_attrs['raw_mae'])

# Standard deviation of MAE across folds
print("MAE Standard Deviation:", study.best_trial.user_attrs['mae_std'])

# Best parameters
print("\nBest parameters:")
print(study.best_params)

# Best trial number
print("\nBest trial number:", study.best_trial.number)

# All trials information
print("\nAll trials:")
trials_df = study.trials_dataframe()
print(trials_df.sort_values('value').head())  # Shows top 5 trials

Best MAE (with variance penalty): 0.5452307215638383
Raw MAE: 0.5200646370615968
MAE Standard Deviation: 0.04839030133721801

Best parameters:
{'booster': 'gbtree', 'lambda': 2.806519745479964e-05, 'alpha': 8.971705154637653e-06, 'eta': 0.010106928111265548, 'gamma': 3.587578552074615e-06, 'max_depth': 29, 'min_child_weight': 13, 'subsample': 0.9986782885481962, 'colsample_bytree': 0.6404365813951769, 'colsample_bylevel': 0.5814819838786656, 'colsample_bynode': 0.858396343604288, 'max_delta_step': 10, 'n_estimators': 1288, 'grow_policy': 'depthwise'}

Best trial number: 93

All trials:
     number     value             datetime_start          datetime_complete  \
93       93  0.545231 2024-12-19 21:09:27.456988 2024-12-19 21:10:39.642080   
73       73  0.545708 2024-12-19 20:58:07.259950 2024-12-19 20:59:16.675232   
144     144  0.545802 2024-12-19 21:19:56.176570 2024-12-19 21:20:54.533170   
142     142  0.546033 2024-12-19 21:19:44.109708 2024-12-19 21:21:02.843436   
143     143 

In [32]:
from xgboost import XGBRegressor

def objective(trial):
    # Create features
    X_train_feat = create_features(X_train)
    X_test_feat = create_features(X_test)
    
    # Define XGBoost parameters
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        'max_depth': trial.suggest_int('max_depth', 3, 30),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.5, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 15),
        'subsample': trial.suggest_float('subsample', 0.3, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.3, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.3, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'gamma': trial.suggest_float('gamma', 1e-8, 10.0, log=True),
        'max_delta_step': trial.suggest_int('max_delta_step', 0, 10),
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        'random_state': 42,
        'n_jobs': -1,
        'early_stopping_rounds': 50  # Moved here
    }
    
    if params['booster'] == 'dart':
        params['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
        params['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
        params['rate_drop'] = trial.suggest_float('rate_drop', 0.0, 0.5)
        params['skip_drop'] = trial.suggest_float('skip_drop', 0.0, 0.5)
    
    # Implement k-fold cross validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mae_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_feat)):
        X_fold_train = X_train_feat.iloc[train_idx]
        y_fold_train = y_train.iloc[train_idx]
        X_fold_val = X_train_feat.iloc[val_idx]
        y_fold_val = y_train.iloc[val_idx]
        
        # Preprocess the data
        X_fold_train_processed, X_fold_val_processed = preprocess_data(
            X_fold_train, X_fold_val, y_fold_train
        )
        
        # Create and train model
        model = XGBRegressor(**params)
        model.fit(
            X_fold_train_processed, 
            y_fold_train,
            eval_set=[(X_fold_val_processed, y_fold_val)],
            verbose=False
        )
        
        # Predict and calculate MAE
        preds = model.predict(X_fold_val_processed)
        mae = mean_absolute_error(np.expm1(y_fold_val), np.expm1(preds))
        mae_scores.append(mae)
    
    mean_mae = np.mean(mae_scores)
    mae_std = np.std(mae_scores)
    final_score = mean_mae * (1 + mae_std)
    
    trial.set_user_attr('mae_std', mae_std)
    trial.set_user_attr('raw_mae', mean_mae)
    
    return final_score

# ... rest of the code remains same until final model training ...

# Train final model
best_params['early_stopping_rounds'] = 50  # Add early stopping to best params
final_model = XGBRegressor(**best_params, random_state=42, n_jobs=-1)
final_model.fit(
    X_train_processed,
    y_train,
    eval_set=[(X_test_processed, y_test)],
    verbose=False
)

# Final evaluation
final_preds = final_model.predict(X_test_processed)
final_mae = mean_absolute_error(np.expm1(y_test), np.expm1(final_preds))
final_r2 = r2_score(y_test, final_preds)

# Feature importance analysis
feature_importance = final_model.feature_importances_
feature_names = [f"feature_{i}" for i in range(X_train_processed.shape[1])]
importance_dict = dict(zip(feature_names, feature_importance))
sorted_importance = sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)

In [35]:
final_mae,final_r2

(np.float64(0.510324515712915), 0.9017210523727519)

In [36]:
def objective(trial):
    # Create features
    X_train_feat = create_features(X_train)
    X_test_feat = create_features(X_test)
    
    # Define XGBoost parameters
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        'max_depth': trial.suggest_int('max_depth', 3, 30),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.5, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 15),
        'subsample': trial.suggest_float('subsample', 0.3, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'random_state': 42,
        'n_jobs': -1,
        'early_stopping_rounds': 50
    }
    
    # Simplified cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mae_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_feat)):
        X_fold_train = X_train_feat.iloc[train_idx]
        y_fold_train = y_train.iloc[train_idx]
        X_fold_val = X_train_feat.iloc[val_idx]
        y_fold_val = y_train.iloc[val_idx]
        
        # Preprocess the data
        X_fold_train_processed, X_fold_val_processed = preprocess_data(
            X_fold_train, X_fold_val, y_fold_train
        )
        
        # Create and train model
        model = XGBRegressor(**params)
        model.fit(
            X_fold_train_processed, 
            y_fold_train,
            eval_set=[(X_fold_val_processed, y_fold_val)],
            verbose=False
        )
        
        # Predict and calculate MAE
        preds = model.predict(X_fold_val_processed)
        mae = mean_absolute_error(np.expm1(y_fold_val), np.expm1(preds))
        mae_scores.append(mae)
    
    return np.mean(mae_scores)  # Return just the mean MAE

# Create study with more trials
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)  # Increased number of trials

# Print the best parameters
print("Best parameters:", study.best_params)
print("Best MAE:", study.best_value)

# Train final model with best parameters
best_params = study.best_params
best_params['early_stopping_rounds'] = 50
final_model = XGBRegressor(**best_params, random_state=42, n_jobs=-1)

# Create final features and preprocess
X_train_feat = create_features(X_train)
X_test_feat = create_features(X_test)
X_train_processed, X_test_processed = preprocess_data(X_train_feat, X_test_feat, y_train)

# Train and evaluate
final_model.fit(
    X_train_processed,
    y_train,
    eval_set=[(X_test_processed, y_test)],
    verbose=False
)

final_preds = final_model.predict(X_test_processed)
final_mae = mean_absolute_error(np.expm1(y_test), np.expm1(final_preds))
print(f"Final MAE: {final_mae:.4f}")

[I 2024-12-22 16:33:57,363] A new study created in memory with name: no-name-89d3ec03-2843-4290-bcc8-e40396f5140c
[I 2024-12-22 16:34:02,196] Trial 0 finished with value: 0.5604361461782889 and parameters: {'n_estimators': 410, 'max_depth': 26, 'learning_rate': 0.14011894507581135, 'min_child_weight': 2, 'subsample': 0.6826774563596791, 'colsample_bytree': 0.5790888087240276, 'reg_alpha': 1.8089168189992698e-08, 'reg_lambda': 1.7911616055420272e-08}. Best is trial 0 with value: 0.5604361461782889.
[I 2024-12-22 16:34:03,666] Trial 1 finished with value: 0.535513141362441 and parameters: {'n_estimators': 1278, 'max_depth': 18, 'learning_rate': 0.13880132132659198, 'min_child_weight': 14, 'subsample': 0.6773880404690773, 'colsample_bytree': 0.5735246420424917, 'reg_alpha': 1.5451762235465848e-05, 'reg_lambda': 7.99103358199112e-07}. Best is trial 1 with value: 0.535513141362441.
[I 2024-12-22 16:34:12,486] Trial 2 finished with value: 0.9893437548837595 and parameters: {'n_estimators': 8

KeyboardInterrupt: 

In [27]:
import numpy as np
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# Convert categorical columns to category dtype
categorical_cols = ['property_type', 'sector', 'balcony', 'agePossession', 
                   'furnishing_type', 'luxury_category', 'floor_category']
for col in categorical_cols:
    X[col] = X[col].astype('category')

# Define objective function for Optuna
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 0.5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1)
    }
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.2, random_state=42)
    
    model = XGBRegressor(
        **params,
        random_state=42,
        enable_categorical=True,
        tree_method='hist',
        missing=np.nan
    )
    
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_test, y_test)],
        early_stopping_rounds=10,
        verbose=False
    )
    
    y_pred = model.predict(X_test)
    return mean_absolute_error(np.expm1(y_test), np.expm1(y_pred))

# Optimize hyperparameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Train final model with best parameters
best_params = study.best_params
final_model = XGBRegressor(
    **best_params,
    random_state=42,
    enable_categorical=True,
    tree_method='hist',
    missing=np.nan
)

X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.2, random_state=42)
final_model.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=10,
    verbose=False
)

# Evaluate final model
y_pred = final_model.predict(X_test)
mae = mean_absolute_error(np.expm1(y_test), np.expm1(y_pred))
print(f'Final MAE: {mae}')

[I 2024-12-22 18:34:54,994] A new study created in memory with name: no-name-1fb3ee30-b6ed-4959-8af8-c64dc9f4480f
[W 2024-12-22 18:34:55,006] Trial 0 failed with parameters: {'n_estimators': 130, 'max_depth': 10, 'learning_rate': 0.06406196892704631, 'min_child_weight': 2, 'subsample': 0.9627888608712818, 'colsample_bytree': 0.9058027646212827, 'gamma': 0.008752130473296371, 'reg_alpha': 0.9963903280345212, 'reg_lambda': 0.9670074373669226} because of the following error: AttributeError('`np.NaN` was removed in the NumPy 2.0 release. Use `np.nan` instead.').
Traceback (most recent call last):
  File "c:\Users\vijit_singh\Desktop\Personal projects repo\Estate-Radar\env\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\vijit_singh\AppData\Local\Temp\ipykernel_25736\4067707982.py", line 39, in objective
    model.fit(
  File "c:\Users\vijit_singh\Desktop\Personal projects repo\Estate-R

AttributeError: `np.NaN` was removed in the NumPy 2.0 release. Use `np.nan` instead.

In [34]:
import optuna
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 800, 3000),
        'max_depth': trial.suggest_int('max_depth', 4, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.03),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        'subsample': trial.suggest_float('subsample', 0.6, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1.0)
    }
    
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', XGBRegressor(
            **params,
            random_state=42,
            enable_categorical=True
        ))
    ])
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(
        model, X, y_transformed, 
        cv=kf,
        scoring='neg_mean_absolute_error'
    )
    
    return -scores.mean()

# Run optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=40)

# Train final model with best parameters
final_model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(
        **study.best_trial.params,
        random_state=42,
        enable_categorical=True
    ))
])

X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.2, random_state=42)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)
final_mae = mean_absolute_error(np.expm1(y_test), np.expm1(y_pred))
print(f"Final MAE: {final_mae}")

[I 2024-12-22 18:50:55,192] A new study created in memory with name: no-name-9281961e-e503-4800-beea-44a4e40e942d
[I 2024-12-22 18:51:04,283] Trial 0 finished with value: 0.11358788450891928 and parameters: {'n_estimators': 2772, 'max_depth': 4, 'learning_rate': 0.025527539329031387, 'min_child_weight': 6, 'subsample': 0.662403821071277, 'colsample_bytree': 0.8375612125258681, 'reg_lambda': 0.4519688816856576, 'reg_alpha': 0.39831423122598686}. Best is trial 0 with value: 0.11358788450891928.
[I 2024-12-22 18:51:16,288] Trial 1 finished with value: 0.11150526570374461 and parameters: {'n_estimators': 998, 'max_depth': 11, 'learning_rate': 0.0057584787511442675, 'min_child_weight': 6, 'subsample': 0.7658337167365619, 'colsample_bytree': 0.848459791239544, 'reg_lambda': 0.8095937037967035, 'reg_alpha': 0.21715816258337528}. Best is trial 1 with value: 0.11150526570374461.
[I 2024-12-22 18:51:26,968] Trial 2 finished with value: 0.11189448573184954 and parameters: {'n_estimators': 1193, '

Final MAE: 0.44413879753668095


In [28]:

columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ], 
    remainder='passthrough'
)

In [29]:
# def scorer(model_name, model):
    
# output = []

# output.append(model_name)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

print(scores.mean())



    

0.9047983252719011


In [30]:
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

pipeline.fit(X_train,y_train)

y_pred = pipeline.predict(X_test)

y_pred = np.expm1(y_pred)

mean_absolute_error(np.expm1(y_test),y_pred)



np.float64(0.447518119423869)